In [4]:
import torch
import torch.nn as nn
from modules.Conv import ConvBlock, DeConvConnectBlock, DeConvBlock
from modules.DPyConv import DPyConv
from torchsummary import summary

In [5]:
class someNet(nn.Module):
    def __init__(self, channels=64) -> None:
        super(someNet, self).__init__()
        self.conv1 = ConvBlock(1, 1 * channels, 3, 2, 1)
        self.pyconv4 = DPyConv(1 * channels)
        self.conv2 = ConvBlock(1 * channels, 2 * channels, 3, 2, 1)
        self.pyconv3 = DPyConv(2 * channels)
        self.conv3 = ConvBlock(2 * channels, 4 * channels, 3, 2, 1)
        self.pyconv2 = DPyConv(4 * channels)
        self.conv4 = ConvBlock(4 * channels, 8 * channels, 3, 2, 1)
        self.pyconv1 = DPyConv(8 * channels)
        self.conv5 = ConvBlock(8 * channels, 16 * channels, 3, 2, 1)
        self.deconv1 = DeConvConnectBlock(8 * channels)
        self.deconv2 = DeConvConnectBlock(4 * channels)
        self.deconv3 = DeConvConnectBlock(2 * channels)
        self.deconv4 = DeConvConnectBlock(1 * channels)
        self.deconv = DeConvBlock(1 * channels, 1, 3, 1, 1)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)
        x4 = self.pyconv1(x4)
        x = self.deconv1(x5,x4)
        x3 = self.pyconv2(x3)
        x = self.deconv2(x,x3)
        x2 = self.pyconv3(x2)
        x = self.deconv3(x,x2)
        x1 = self.pyconv4(x1)
        x = self.deconv4(x,x1)
        x = self.deconv(x)
        return x

In [6]:
model = someNet()
a = summary(model, (1,320,640))

Layer (type:depth-idx)                   Output Shape              Param #
├─ConvBlock: 1-1                         [-1, 64, 160, 320]        --
|    └─Conv2d: 2-1                       [-1, 64, 320, 640]        576
|    └─BatchNorm2d: 2-2                  [-1, 64, 320, 640]        128
|    └─MaxPool2d: 2-3                    [-1, 64, 160, 320]        --
|    └─ReLU: 2-4                         [-1, 64, 160, 320]        --
├─ConvBlock: 1-2                         [-1, 128, 80, 160]        --
|    └─Conv2d: 2-5                       [-1, 128, 160, 320]       73,728
|    └─BatchNorm2d: 2-6                  [-1, 128, 160, 320]       256
|    └─MaxPool2d: 2-7                    [-1, 128, 80, 160]        --
|    └─ReLU: 2-8                         [-1, 128, 80, 160]        --
├─ConvBlock: 1-3                         [-1, 256, 40, 80]         --
|    └─Conv2d: 2-9                       [-1, 256, 80, 160]        294,912
|    └─BatchNorm2d: 2-10                 [-1, 256, 80, 160]        512
| 